# CNN with mnist using subclassing API

In [1]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__) # find the version number (should be 2.x+)

# 그래피카드 유무 확인 및 메모리 확장 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  print('사용가능한 GPU 갯수: ',len(gpus), '\n')
      
  try:
    # 프로그램이 실행되어 더 많은 GPU 메모리가 필요하면, 텐서플로 프로세스에 할당된 GPU 메모리 
    # 영역을 확장할 수있도록 허용
    tf.config.experimental.set_memory_growth(gpus[0], True)

  except RuntimeError as e:
    # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
    print(e)

# 설치된 GPU 상세내용 확인
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2.6.0
사용가능한 GPU 갯수:  1 

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14823475340577296736
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 16185556992
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11765833760561082667
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


## tf.data 사용법

TensorFlow에서는 Dataset 이라는 built-in-API를 제공하고 있어서 위의 작업을 쉽게 처리할 수 있다. 이 포스트에서는 입력 파이프라인을 만들어서 모델에 데이터를 효율적으로 공급하는 방법을 살펴볼 것이다. 또한, 흔하게 볼 수 있는 예시를 다루면서 Dataset의 기본적인 메커니즘을 설명할 것이다.

Dataset을 사용하려면 세 가지 단계를 거쳐야한다.

  - 데이터셋 생성하기: 사용하려는 데이터로부터 Dataset 인스턴스를 만든다.
  - Iterator(반복자) 생성하기. 생성된 데이터를 사용해서 Iterator 인스턴스를 만들어 Dataset을 iterate시킨다.
  - 데이터 사용하기. 생성된 iterator를 사용해서 모델에 공급할 dataset으로부터 요소를 가져올 수 있다.

### 데이터 불러오기/Iterator(반복자) 생성하기

일단 dataset안에 넣을 데이터가 필요하다.

  - list 
  - numpy array
  - tensor

__from_tensor_slices()__

It creates a Dataset whose elements are slices of the given tensors.

  > 
  > from_tensor_slices(tensors)
  >

The given tensors are sliced along their first dimension. This operation preserves the structure of the input tensors, removing the first dimension of each tensor and using it as the dataset dimension. All input tensors must have the same size in their first dimensions.


In [2]:
# list에서 불러오기

ds = tf.data.Dataset.from_tensor_slices([[1,2,3,4,5],
                                         [5,6,7,8,9]])

ds

<TensorSliceDataset shapes: (5,), types: tf.int32>

In [3]:
import numpy as np
# numpy에서 불러오기
# create a random vector of shape (10,2)
x = np.random.sample((10,2))
# make a dataset from a numpy array
ds = tf.data.Dataset.from_tensor_slices(x)

print(type(ds))
list(ds.as_numpy_iterator())

<class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>


[array([0.28742414, 0.74769832]),
 array([0.65408172, 0.18287757]),
 array([0.19821726, 0.4241679 ]),
 array([0.85702003, 0.45553447]),
 array([0.26763705, 0.61746745]),
 array([0.77676629, 0.38342378]),
 array([0.29489216, 0.98122835]),
 array([0.11567539, 0.85496447]),
 array([0.80541762, 0.70710743]),
 array([0.47871613, 0.83781296])]

또한 데이터를 특성(feature)과 라벨(label)로 나누어 사용하는 경우처럼, 한 개 이상의 numpy 배열을 넣을 수도 있다.

In [4]:
features, labels = (np.random.sample((10,2)), np.random.sample((10,1)))
ds = tf.data.Dataset.from_tensor_slices((features,labels))

print(type(ds))
list(ds.as_numpy_iterator())

<class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>


[(array([0.59909138, 0.68485113]), array([0.40174538])),
 (array([0.62649744, 0.75297804]), array([0.35082164])),
 (array([0.3102005 , 0.95268443]), array([0.37910474])),
 (array([0.12716688, 0.23973209]), array([0.41891797])),
 (array([0.96790375, 0.45892965]), array([0.76406124])),
 (array([0.14100629, 0.46028756]), array([0.077457])),
 (array([0.25091007, 0.03379486]), array([0.19465404])),
 (array([0.67369105, 0.75833845]), array([0.31280578])),
 (array([0.45376893, 0.57216387]), array([0.05294739])),
 (array([0.43755944, 0.84518217]), array([0.10318423]))]

In [5]:
# tensor에서 불러오기
ds = tf.data.Dataset.from_tensor_slices(tf.random.uniform([100, 2]))

ds

<TensorSliceDataset shapes: (2,), types: tf.float32>

### as_numpy_iterator() 

It returns an iterable over the elements of the dataset, with their tensors converted to numpy arrays.

In [6]:
for num in ds.as_numpy_iterator():
  print(num)

[0.06076014 0.75210893]
[0.5227314  0.38314462]
[0.37224174 0.33730292]
[0.6399156 0.8571887]
[0.13419497 0.31620705]
[0.53104246 0.96876264]
[0.77490366 0.8695797 ]
[0.4010228  0.46247923]
[0.2701255 0.4553125]
[0.03831863 0.46635127]
[0.6446265 0.6346551]
[0.5888603  0.22811854]
[0.6593696 0.1938585]
[0.72324014 0.8723382 ]
[0.06202817 0.78531754]
[0.6805862 0.7266767]
[0.80385864 0.28127277]
[0.31983554 0.85509396]
[0.48077726 0.38749194]
[0.2623979  0.24601924]
[0.4366157  0.48160756]
[0.15135455 0.25945437]
[0.7980974  0.21606481]
[0.6158446  0.62218463]
[0.36767685 0.31302786]
[0.6627089 0.9398912]
[0.87256455 0.7167152 ]
[0.5833521 0.8268455]
[0.5393417  0.05451369]
[0.7467134 0.9027438]
[0.70380163 0.6689832 ]
[0.09288752 0.41362727]
[0.21158862 0.6166409 ]
[0.21115589 0.24647295]
[0.3171779 0.0538553]
[0.77096796 0.47211123]
[0.93307364 0.6676769 ]
[0.78123343 0.5420424 ]
[0.20008767 0.8474612 ]
[0.1856848  0.19465673]
[0.13271916 0.39921916]
[0.38916755 0.9233011 ]
[0.1586275

### batch(): 

  It combines consecutive elements of this dataset into batches.

  > 
  > batch(batch_size, drop_remainder=False)
  >
  

In [7]:
ds = tf.data.Dataset.range(10)
ds = ds.batch(3)
list(ds.as_numpy_iterator())

[array([0, 1, 2]), array([3, 4, 5]), array([6, 7, 8]), array([9])]

In [8]:
ds = tf.data.Dataset.range(10)
ds = ds.batch(3, drop_remainder=True)
list(ds.as_numpy_iterator())

[array([0, 1, 2]), array([3, 4, 5]), array([6, 7, 8])]

### filter()

Filters this dataset according to predicate.

  >
  > filter(predicate)
  >

In [9]:
ds = tf.data.Dataset.from_tensor_slices([1, 2, 3,4,5])
ds = ds.filter(lambda x: x < 3)
list(ds.as_numpy_iterator())

[1, 2]

In [10]:
def filter_fn(x):
  return tf.math.less(x, 3)

ds = tf.data.Dataset.from_tensor_slices([1,2,3,4,5])
ds = ds.filter(filter_fn)
list(ds.as_numpy_iterator())

[1, 2]

### apply()

Applies a transformation function to this dataset.

  >
  > apply(transformation_func)
  >

In [11]:
dataset = tf.data.Dataset.range(100)
def dataset_fn(ds):
  return ds.filter(lambda x: x < 10)
ds = dataset.apply(dataset_fn)
list(ds.as_numpy_iterator())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### shuffle()

  >
  > shuffle(buffer_size, seed=None, reshuffle_each_iteration=None)
  >

It randomly shuffles the elements of this dataset.

This dataset fills a buffer with buffer_size elements, then randomly samples elements from this buffer, replacing the selected elements with new elements. For perfect shuffling, a buffer size greater than or equal to the full size of the dataset is required.

In [12]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

## CNN with mnist using subclassing API  

[MNIST 데이터셋](http://yann.lecun.com/exdb/mnist/)을 로드하여 준비합니다.

In [13]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 채널 차원을 추가합니다.
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

tf.data를 사용하여 데이터셋을 섞고 배치를 만듭니다:

In [14]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

### 모델 서브클래싱(subclassing) API

케라스(Keras)의 [모델 서브클래싱(subclassing) API](https://www.tensorflow.org/guide/keras#model_subclassing)를 사용하여 `tf.keras` 모델을 만듭니다:

In [15]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

훈련에 필요한 옵티마이저(optimizer)와 손실 함수를 선택합니다:

In [16]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

모델의 손실과 성능을 측정할 지표를 선택합니다. 에포크가 진행되는 동안 수집된 측정 지표를 바탕으로 최종 결과를 출력합니다.

In [17]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy') # Calculates how often predictions match integer labels.

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

`tf.GradientTape`를 사용하여 모델을 훈련합니다:

In [18]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

이제 모델을 테스트합니다:

In [19]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [20]:
EPOCHS = 20

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

에포크: 1, 손실: 0.13577836751937866, 정확도: 95.79166412353516, 테스트 손실: 0.05823395028710365, 테스트 정확도: 98.02999877929688
에포크: 2, 손실: 0.08848886936903, 정확도: 97.26333618164062, 테스트 손실: 0.05722919851541519, 테스트 정확도: 98.13999938964844
에포크: 3, 손실: 0.06612000614404678, 정확도: 97.94944763183594, 테스트 손실: 0.05684208869934082, 테스트 정확도: 98.20333099365234
에포크: 4, 손실: 0.052878715097904205, 정확도: 98.35208129882812, 테스트 손실: 0.060180000960826874, 테스트 정확도: 98.11750030517578
에포크: 5, 손실: 0.04417392984032631, 정확도: 98.61566162109375, 테스트 손실: 0.06147143617272377, 테스트 정확도: 98.14399719238281
에포크: 6, 손실: 0.03791283816099167, 정확도: 98.80999755859375, 테스트 손실: 0.06391113996505737, 테스트 정확도: 98.15833282470703
에포크: 7, 손실: 0.03335427865386009, 정확도: 98.95238494873047, 테스트 손실: 0.06614696234464645, 테스트 정확도: 98.16285705566406
에포크: 8, 손실: 0.029646120965480804, 정확도: 99.06791687011719, 테스트 손실: 0.0699506551027298, 테스트 정확도: 98.13999938964844
에포크: 9, 손실: 0.026845399290323257, 정확도: 99.1540756225586, 테스트 손실: 0.07087111473083496, 테스트 정확도: 98

훈련된 이미지 분류기는 이 데이터셋에서 약 98%의 정확도를 달성합니다. 

### @tf.function으로 성능 향상하기

@tf.fuctnion을 이해하기 위해서는 tensorflow 개발과정의 역사(?)를 조금 이해하셔야하는데요.
tf 1.x 버전대에서는 그래프의 생성과 실행을 분리하고 값을 실행할때는 Session이라는 것을 열어서 값을 실행하는 형태였는데요. 이렇게 진행하다보니 값을 계산하고 싶을때마다 Session을 이용해서 실행을 해주어야만해서 프로그래밍 과정상에 불편함이 많았습니다.
따라서 tf 2.x 버전대에서는 Session을 삭제하고 바로 값을 실행할 수 있는 Eager Execution이라는 것이 적용되었는데요. 따라서 값을 계산할때 별도의 Session을 열지 않고도 편리하게 진행할 수 있게 되었습니다.
그럼 왜 굳이 tf 1.x대에서는 저렇게 복잡하게 그래프의 생성과 실행을 분리했느냐라고 생각해 볼 수 있는데요. 해당 형태가 성능상(=속도)의 이점이 있기 때문입니다.

여기서 @tf.function 관련된 내용이 나오게 되는데요. def 위에 @tf.fucnction을 붙이면 마치 tf2.x 버전에서도 tf1.x 형태처럼 그래프 생성과 실행이 분리된 형태로 해당 함수내의 로직이 실행되게 됩니다. 따라서 상황에 따라서 성능이 약간 향상 될 수 있는데요.(=실행 속도가 약간 빨라질 수 있습니다.) 다만 해당 annoation을 붙이면 tf1.x처럼 해당 함수내의 값을 바로 계산해볼수 없어서 디버깅이 불편해질수 있습니다. 따라서 모든 로직에 대한 프로그래밍이 끝난 상태에서 @tf.fuction을 붙이는 것이 좋습니다.

즉 정리하면
1. @tf.fucntion을 붙이면 tf1.x 스타일로 해당 함수내의 로직이 동작한다.
2. 따라서 상황에 따라 속도가 약간 빨라질 수 있다.
3. 다만 해당 annotation을 붙이면 값을 바로 계산해볼수 없어서 모든 로직에 대한 프로그래밍이 끝난 뒤에 붙이는 것이 좋다.

In [21]:
@tf.function
def add(a, b):
  print(a,b)
  # print('tf.variable 1 =',tf.Variable(3))
  return a + b

print('tf.variable 2 =',tf.Variable(3))
add(tf.ones([2, 2]), tf.ones([2, 2]))  #  [[2., 2.], [2., 2.]]

tf.variable 2 = <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=3>
Tensor("a:0", shape=(2, 2), dtype=float32) Tensor("b:0", shape=(2, 2), dtype=float32)


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [22]:
def add(a, b):
  print(a,b)
  print('tf.variable 1 =',tf.Variable(3))
  return a + b

print('tf.variable 2 =',tf.Variable(3))
add(tf.ones([2, 2]), tf.ones([2, 2]))  #  [[2., 2.], [2., 2.]]

tf.variable 2 = <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=3>
tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32) tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)
tf.variable 1 = <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=3>


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [23]:
v = tf.Variable(1.0)
with tf.GradientTape() as tape:
  result = add(v, 1.0)
tape.gradient(result, v)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0> 1.0
tf.variable 1 = <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=3>


<tf.Tensor: shape=(), dtype=float32, numpy=1.0>